# Package Installation
First, let's install the required Python packages using pip.

In [5]:
%pip install pandas numpy matplotlib seaborn plotly

1107.44s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/14.8 MB ? eta -:--:--Downloading plotly-6.0.1-py3-none-any.whl (14.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 180.3 kB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 180.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.4/338.4 kB 206.3 kB/s eta 0:00:00a 0:00:01━━━━━━━━━━━━━━━━━━━━━━ 0.0/338.4 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.4/338.4 kB 206.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# COVID-19 Global Data Analysis

This notebook presents a comprehensive analysis of global COVID-19 data, including cases, deaths, and vaccination progress.

In [6]:
import sys
sys.path.append('../src')
from data_loader import CovidDataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Initialize and load data
loader = CovidDataLoader()
df = loader.load_data()
df_clean = loader.clean_data()

INFO:data_loader:Downloading data from web...
ERROR:data_loader:Error loading data: IncompleteRead(102694416 bytes read, 195015 more expected)


IncompleteRead: IncompleteRead(102694416 bytes read, 195015 more expected)

## Key Insights
Let's examine the latest statistics and trends across countries.

In [ ]:
insights = loader.get_key_insights()
print(f"As of {insights['date']}, {insights['highest_cases']['country']} leads in total cases")

# Display all visualizations
loader.plot_total_cases()
loader.plot_new_cases_trend()
loader.plot_vaccination_progress()

## Growth Rate Analysis
Calculate and analyze the case doubling time for each country during the first 90 days of their pandemic.

In [ ]:
def calculate_doubling_time(country_name, start_date, end_date):
    country_data = df_clean[(df_clean['location'] == country_name) & 
                           (df_clean['date'] >= start_date) & 
                           (df_clean['date'] <= end_date)]
    if len(country_data) < 2:
        return np.nan
    cases = country_data['total_cases'].values
    days = len(country_data)
    growth_rate = (cases[-1] / cases[0]) ** (1/days) - 1
    if growth_rate <= 0:
        return np.inf
    return np.log(2) / np.log(1 + growth_rate)

# Calculate doubling time for each country
for country in loader.countries_of_interest:
    country_data = df_clean[df_clean['location'] == country]
    start_date = country_data['date'].min()
    end_date = start_date + pd.Timedelta(days=90)
    doubling_time = calculate_doubling_time(country, start_date, end_date)
    print(f"{country} had a case doubling time of {round(doubling_time, 1)} days in the first 90 days.")

## Correlation Analysis
Examine relationships between different COVID-19 metrics and socioeconomic indicators.

In [ ]:
# Get latest data for correlation analysis
latest_data = loader.get_latest_data()
corr_data = latest_data[['total_cases', 'total_deaths', 'total_vaccinations', 
                         'population', 'gdp_per_capita']]

# Create correlation matrix
corr_matrix = corr_data.corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of COVID-19 Metrics', fontsize=16)
plt.tight_layout()
plt.show()